In [26]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from skimage.io import imread
from skimage.transform import resize
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import csv
tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [2]:
IMG_SIZE = 100
DS_SIZE = 6000
EPS = 1e-8
BATCH_SIZE = 30

In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [4]:
get_available_gpus()

['/device:GPU:0']

In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
# from tensorflow.keras.layers import Dense, Activation, ReLU, Conv2D, MaxPool2D, BatchNormalization, Dense

In [6]:
def load_and_resize(filename, Y, num):
    image = imread(filename)
    hei = image.shape[0]
    wid = image.shape[1]
    Y[i,::2] = (Y[i,::2] * 100) / wid
    Y[i,1::2] = (Y[i,1::2] * 100) / hei
    image = resize(image, (IMG_SIZE, IMG_SIZE))
    if len(image.shape) == 2:
        image = np.stack((image, image, image))
        image = np.transpose(image, axes=(1,2, 0))
#     print(image.shape)
    for channel in range(3):
        image[:,:,channel] = (image[:,:,channel] - np.mean(image[:,:,channel])) / (np.std(image[:,:,channel]) + EPS)
#     print(image.shape)
    return image

In [7]:
def read_gt():
#     csv_file = csv.DictReader(open('public_tests/00_test_img_input/train/gt.csv'))
#     lines = [line for line in csv_file]
    lines = pd.read_csv('public_tests/00_test_img_input/train/gt.csv')
    lines.drop('filename', axis=1, inplace=True)
    return lines

In [8]:
# lines = read_gt()[:100]
# print(lines)

In [9]:
# for i in range(200, 300):
#     print(i)
#     X[i] = 
# img = load_and_resize('public_tests/00_test_img_input/train/images/{:05}.jpg'.format(20))
# print(img.shape)
# print(img[0, 0, :])

In [10]:
X = np.zeros((DS_SIZE, IMG_SIZE, IMG_SIZE, 3))
Y = read_gt()[:DS_SIZE].values.astype(float)
for i in range(DS_SIZE):
#     print(Y[i])
    X[i] = load_and_resize('public_tests/00_test_img_input/train/images/{:05}.jpg'.format(i), Y, i)
#     print(Y[i])
    if i % 100 == 0:
        print('{} loaded'.format(i))
#     print('public_tests/00_test_img_input/train/images/{:05}.jpg'.format(i))
    

0 loaded
100 loaded
200 loaded
300 loaded
400 loaded
500 loaded
600 loaded
700 loaded
800 loaded
900 loaded
1000 loaded
1100 loaded
1200 loaded
1300 loaded
1400 loaded
1500 loaded
1600 loaded
1700 loaded
1800 loaded
1900 loaded
2000 loaded
2100 loaded
2200 loaded
2300 loaded
2400 loaded
2500 loaded
2600 loaded
2700 loaded
2800 loaded
2900 loaded
3000 loaded
3100 loaded
3200 loaded
3300 loaded
3400 loaded
3500 loaded
3600 loaded
3700 loaded
3800 loaded
3900 loaded
4000 loaded
4100 loaded
4200 loaded
4300 loaded
4400 loaded
4500 loaded
4600 loaded
4700 loaded
4800 loaded
4900 loaded
5000 loaded
5100 loaded
5200 loaded
5300 loaded
5400 loaded
5500 loaded
5600 loaded
5700 loaded
5800 loaded
5900 loaded


In [11]:
print(X.shape)

(6000, 100, 100, 3)


In [12]:
print(Y)

[[19.46564885 19.46564885 48.47328244 ... 68.70229008 64.1221374
  68.32061069]
 [22.34042553 21.27659574 38.29787234 ... 70.21276596 63.82978723
  69.14893617]
 [13.85542169 69.87951807 23.4939759  ... 60.24096386 72.89156627
  48.79518072]
 ...
 [13.46153846 21.15384615 26.28205128 ... 73.07692308 60.25641026
  71.15384615]
 [15.68627451 25.49019608 33.33333333 ... 68.62745098 66.66666667
  66.66666667]
 [19.14893617 17.0212766  36.17021277 ... 70.21276596 68.08510638
  68.08510638]]


In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [27]:
model = keras.Sequential()
model.add(layers.Conv2D(64, kernel_size=(3,3), input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPool2D())

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
# model.add(layers.BatchNormalization())
model.add(layers.Dense(28))

In [28]:
train_datagen = ImageDataGenerator()
train_datagen.fit(x_train)
train_generator = train_datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)
step = x_train.shape[0] // BATCH_SIZE

In [29]:
adam = keras.optimizers.Adam(learning_rate=0.005)

In [30]:
def custom_loss(layer):
    
    def loss(y_true, y_pred):
        return K.mean(K.square())

In [31]:
model.compile(optimizer=adam,loss='mean_squared_error', metrics=['accuracy'])

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 98, 98, 64)        256       
_________________________________________________________________
re_lu (ReLU)                 (None, 98, 98, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 47, 47, 128)       512       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 47, 47, 128)       0

In [33]:
history = model.fit_generator(train_generator, steps_per_epoch=step, epochs=10,
                              validation_data=(x_test, y_test))

Epoch 1/10
160/160 [==============================] - 24s 147ms/step - loss: 119.6992 - accuracy: 0.6402 - val_loss: 35.8962 - val_accuracy: 0.6100
Epoch 2/10
160/160 [==============================] - 23s 146ms/step - loss: 22.9114 - accuracy: 0.7554 - val_loss: 42.8321 - val_accuracy: 0.6075
Epoch 3/10
160/160 [==============================] - 23s 146ms/step - loss: 19.4324 - accuracy: 0.7571 - val_loss: 60.3941 - val_accuracy: 0.7817
Epoch 4/10
160/160 [==============================] - 23s 145ms/step - loss: 17.4547 - accuracy: 0.7552 - val_loss: 20.8389 - val_accuracy: 0.8158
Epoch 5/10
160/160 [==============================] - 23s 146ms/step - loss: 14.3456 - accuracy: 0.8004 - val_loss: 23.1839 - val_accuracy: 0.8333
Epoch 6/10
160/160 [==============================] - 23s 146ms/step - loss: 13.5558 - accuracy: 0.8000 - val_loss: 22.1747 - val_accuracy: 0.8375
Epoch 7/10
160/160 [==============================] - 23s 145ms/step - loss: 12.6988 - accuracy: 0.8206 - val_loss: 3

In [42]:
y_pred = model.predict(x_test)

In [43]:
def compute_metric(pred, test):
    res = 0.0
    hei = pred.shape[0]
    print(pred.shape)
    for i in range(hei):
        diff = (pred[i] - test[i])
#         diff[::2] /= n_cols
#         diff[1::2] /= n_rows
#         diff *= 100
#         print((diff ** 2).mean())
        res += (diff ** 2).mean()
    return res / hei

print(compute_metric(y_pred, y_test))

(1200, 28)
18.48273013649225


In [49]:
for line in range(50):
#     print(y_pred[line])
#     print(y_test[line])
    print((np.abs(y_pred[line] - y_test[line])).astype(int))

[50 42 10 49 16 47 51 35 39 36 27 35 15 36 20 32 32 33 45 31  1  0 28 24
  0 28 24 26]
[0 1 0 0 0 0 2 0 0 0 0 1 0 2 5 2 4 1 5 1 1 2 1 2 1 6 0 3]
[0 1 0 2 2 1 5 0 2 0 2 1 1 0 0 2 2 1 3 0 1 3 4 0 0 0 0 2]
[2 0 1 3 2 1 7 7 2 0 4 0 5 0 5 2 7 2 7 2 1 2 1 2 0 3 1 0]
[1 1 2 2 0 1 3 5 3 0 2 0 0 2 5 1 1 1 0 2 1 1 3 0 0 0 0 1]
[1 0 0 2 0 0 1 0 2 0 1 0 5 0 1 1 2 3 2 2 2 4 3 1 2 4 2 2]
[1 2 1 0 3 2 4 3 2 0 2 2 1 2 0 1 0 0 2 0 0 0 0 0 0 3 1 1]
[5 3 2 4 2 5 3 3 7 0 6 2 3 2 1 1 0 0 1 1 2 3 0 7 1 2 1 2]
[2 3 3 2 3 1 0 0 0 0 1 0 1 3 2 2 1 1 0 0 0 3 0 1 1 1 3 0]
[1 2 2 0 3 1 2 2 0 1 4 2 0 0 1 0 0 2 1 0 1 2 1 0 1 2 4 3]
[2 0 0 1 2 0 1 5 2 1 4 0 2 4 0 5 1 1 0 3 3 2 2 2 2 4 0 5]
[2 0 2 3 3 2 2 0 1 1 1 1 1 2 0 1 0 2 0 1 0 6 3 1 0 0 0 1]
[3 3 5 0 5 0 4 2 0 1 0 2 2 1 3 3 0 2 0 1 3 2 2 0 0 1 1 2]
[ 5  3  6  6 10  0 16  2  7  4 10  5  7  4 11  0 17  1 17  0  0  5  1  3
  0  3  1  9]
[ 9 14  8  9  8  2 15 10  1 12  3 11  8  7 10  2 12  4  9 11  1  2  0  0
  5  4  2 14]
[2 1 3 1 1 1 0 4 2 0 2 1 0 0 0 1 1 0 2 0 1 